In [5]:
from clustering_tools import *


In [6]:
class AugmentedGraph:

    def __init__(self, nb_cluster):
        self._nb_cluster = nb_cluster 
        self._clusters = None

    def get_clusters(self, _adj_attrs = None, nb_cluster = None):
        if nb_cluster is None :
            nb_cluster = self._nb_cluster
        sc = SpectralClustering(n_clusters=nb_cluster, affinity= "precomputed")
        mod = sc.fit_predict(_adj_attrs)
        nb_nodes = _adj_attrs.shape[0]
        clusters = {
            i : [] for i in range(nb_cluster)
        }
        for i in range(nb_nodes) :
            clusters[mod[i]].append(i)
        return clusters
        
    def get_final_action(self, _selected_cluster, MAX_DEFAULT_NODE_DELAY, _vnf_obs = None, choice = "Min_DELAY"):
        nodes = self._clusters[_selected_cluster]
        _selected = self.select_nod_cluster(nodes, self._costs, self._pns_attrs, MAX_DEFAULT_NODE_DELAY, choice, _vnf_obs = _vnf_obs)
        return _selected
    
    def get_augmented_graph(self, _adj_attrs = None, _pns_attrs = None, keep_clusters = True, _costs = False, MAX_DEFAULT_NODE_DELAY = False, choice = "Max_RAM"):
        
        self._pns_attrs = _pns_attrs
        self._adj_attrs = _adj_attrs
        self._costs = _costs

        if (not keep_clusters) or (self._clusters is None):
            self._clusters = self.get_clusters(_adj_attrs = _adj_attrs)
        _cls_costs = []
        _cls_attrs = None
        for _id, cls in self._clusters.items():
            _selected = self.select_nod_cluster(cls,_costs, _pns_attrs, MAX_DEFAULT_NODE_DELAY, choice)
            _cl_attrs = _pns_attrs[_selected, :]
            if _cls_attrs is None :
                    _cls_attrs = _cl_attrs
            else :
                    _cls_attrs = np.vstack((_cls_attrs, _cl_attrs))

            
            _cls_costs.append(_costs[_selected])
        _cls_costs = np.array(_cls_costs)

        return _cls_attrs, _cls_costs
    
    def select_nod_cluster(self, cls, _costs, _pns_attrs, MAX_DEFAULT_NODE_DELAY, choice, _vnf_obs = None):
        _eligibles = [cls[i] for i in np.where(_costs[cls] < MAX_DEFAULT_NODE_DELAY)[0]]
        if _vnf_obs :
            final = []
            for j in _eligibles :
                if (_vnf_obs[0] <= _pns_attrs[j, 0]) and  (_vnf_obs[1] <= _pns_attrs[j, 1]):
                    final.append(j)
            _eligibles = final 

        if _eligibles: 
            if choice == "Max_RAM":
                # We will take RAM because it's the most critical resource
                _attrs_eligibles = _pns_attrs[_eligibles, :]
                _selected = _eligibles[np.argmax(_attrs_eligibles[: ,1])]
            elif choice == "Min_DELAY":
                # Choose the nearest node as cluster representative 
                _selected = _eligibles[np.argmin(_costs[_eligibles])]

        else:
            _selected = random.choice(cls)
        return _selected 

In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
#Parameteres to store in a config File :

DEFAULT_PN = {
    "Edge" : {
        "CPU" : [7],
        "RAM" : [8192],
    },
    "Transport" : {
        "CPU" : [7],
        "RAM" : [8192],
    },
    "Core" : {
        "CPU" : [7],
        "RAM" : [8192],
    }
}

DEFAULT_PL = {
    "Edge" : {
        "BW" : [50],
        "Delay" : [3],
    },
    "Transport" : {
        "BW" : [50],
        "Delay" : [2.5],
    },
    "Core" : {
        "BW" :  [50],
        "Delay" : [2],
    }
}

DEFAULT_VNF = {
    "CPU" : [0.05, 1],
    "RAM" : [64, 1024],
}

DEFAULT_VL = {
    "BW" : [2,3,4],
    "Delay" : [2,5]
}



In [4]:
dispo = {
 'cluster-c1': {'RAM': 2861.81640625, 'CPU': 2.2299888390269187},
 'cluster-c2': {'RAM': 2920.70703125, 'CPU': 2.378481268636885},
 'cluster-c3': {'RAM': 4916.9765625, 'CPU': 2.7394677932832834},
 'cluster-c4': {'RAM': 4339.5703125, 'CPU': 2.7404578535238193},
 'cluster-c5': {'RAM': 4327.640625, 'CPU': 2.7412230010117176},
 'cluster-c6': {'RAM': 4335.30859375, 'CPU': 2.7307449007273683},
 'cluster-c7': {'RAM': 4356.19921875, 'CPU': 2.7263802633370795},
 'cluster-c8': {'RAM': 4370.375, 'CPU': 2.7438676358997673},
 'cluster-c9': {'RAM': 4347.39453125, 'CPU': 2.740566194880344},
 'cluster-c10': {'RAM': 4347.39453125, 'CPU': 2.740566194880344},
 'cluster-c11': {'RAM': 4347.39453125, 'CPU': 2.740566194880344}
 }

In [5]:
def matrix_to_sparse_adjacency(matrix):
    sparse_adjacency = []
    
    for i in range(len(matrix)):
        for j in range(i + 1, len(matrix[i])):
            if matrix[i][j] != 0:
                sparse_adjacency.append([i,j])
    
    return sparse_adjacency

In [13]:
def define_topology():
    adj_list = [
    [1,2],
    [3],
    [3,4,6],
    [6,7],
    [5],
    [6],
    [7],
    [8,10]
    ,[9]
    ,[10],
      [] ]

    num_nodes = len(adj_list)
    adj_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for node, neighbors in enumerate(adj_list):
        for neighbor in neighbors:
            adj_matrix[node, neighbor] = 1
            adj_matrix[neighbor, node ] = 1

    nsparse_adj_list = matrix_to_sparse_adjacency(adj_matrix)

    nodes_type = [1, 0, 0, 2, 1, 0, 2, 2, 0, 1, 0]
    edges_clusters = {  0 : [0, 4, 9] }
    _adj_attrs = np.zeros((num_nodes, num_nodes, 2))
    _adj_attrs[:, :, 0] = 100
    _PLS = {}
    for j in range(len(nsparse_adj_list)):
        src , dist = nsparse_adj_list[j]
        _type = None 
        if (nodes_type[src] == 0 and nodes_type[dist] == 1) or (nodes_type[src] == 1 and nodes_type[dist] == 0):
            _type = "Edge"
        elif (nodes_type[src] == 0 and nodes_type[dist] == 0) or (nodes_type[src] == 0 and nodes_type[dist] == 2) or (nodes_type[src] == 2 and nodes_type[dist] == 0): 
            _type = "Transport"
        elif (nodes_type[src] == 2 and nodes_type[dist] == 2) : 
            _type = "Core"

        _adj_attrs[src, dist, 1] = random.choice(DEFAULT_PL[_type]["Delay"])
        _adj_attrs[dist, src, 1] = random.choice(DEFAULT_PL[_type]["Delay"])
    return adj_matrix, nsparse_adj_list, _adj_attrs

In [14]:
adj_matrix, nsparse_adj_list, _adj_attrs  = define_topology()

In [15]:

MAX_DEFAULT_NODE_DELAY = 20.0 #The Fixed max used on training

_domain = Domain(0, "0", adj_matrix, nsparse_adj_list, None, None, _adj_attrs = _adj_attrs,
    _deployed = True)

_results = _domain.dijkstra(0, mask_delay = True, 
    MAX_DEFAULT_NODE_DELAY = MAX_DEFAULT_NODE_DELAY,
    required_delay = 15,
    )

_costs = _results[0]

In [16]:
AG = AugmentedGraph(4)

In [28]:
_infra_attributes = [[2.777315411714415, 4383.76953125], [2.8021277875053627, 4489.11328125], [2.8052183805560613, 4492.3515625], [2.8049489519275066, 4503.7265625], [2.8009662854498765, 4455.48828125], [2.81397064753907, 4508.56640625], [2.8044122566358083, 4508.578125], [2.7967584759764166, 4485.734375], [2.8129406418348912, 4478.32421875], [2.7919979551451166, 4488.05078125], [2.8819404255626733, 4497.85546875]]

In [29]:
_cls_attrs, _cls_costs = AG.get_augmented_graph(_adj_attrs = _adj_attrs[:,:,1], _pns_attrs = np.array(_infra_attributes), _costs= _costs, MAX_DEFAULT_NODE_DELAY=  MAX_DEFAULT_NODE_DELAY, choice = "Min_DELAY", keep_clusters= True)

In [39]:
np.hstack((_cls_attrs,  _cls_costs.reshape(-1, 1))).tolist()

[[2.8009662854498765, 4455.48828125, 6.0],
 [2.8049489519275066, 4503.7265625, 5.5],
 [2.8129406418348912, 4478.32421875, 10.0],
 [2.777315411714415, 4383.76953125, 0.0]]

In [40]:
AG.get_final_action(0, MAX_DEFAULT_NODE_DELAY= MAX_DEFAULT_NODE_DELAY, choice = "Max_RAM", _vnf_obs = obs["Vnf"])

NameError: name 'obs' is not defined

In [32]:
_costs.reshape(-1, 1)

array([[ 0. ],
       [ 3. ],
       [ 3. ],
       [ 5.5],
       [ 6. ],
       [ 8. ],
       [ 5.5],
       [ 7.5],
       [10. ],
       [13. ],
       [10. ]])